In [2]:

import os
import pandas as pd
import json

from model import num2cate, generate_samples, generate_model_samples, ModelGene, findKeyAttrs, FindGroups, get_numAttrs, find_rules
from model.samples import DataGene
from model.data_encoder import DataEncoder
from joblib import dump, load


store_path = '../../front/src/testdata/'


def get_rules(dataset_name, protect_attr='',model_name=None):
    
    model_name = '{}_{}'.format(dataset_name, model_name)
    sample_path = os.path.join(store_path, '{}_samplesR.csv'.format(model_name))
    model_samples = pd.read_csv(sample_path)
    rules = find_rules(model_samples, minimum_support=5, min_len=1, protect_attr = protect_attr, target_attr='class', elift_th=[1, 1])

    
    rules.to_json(store_path + '{}_rules.json'.format(model_name),orient='records')

    
def get_samples(dataset_name, protect_attr):
    """
    train model on the training data,
    return the generated samples based on the training data
    """

    sample_num = 1000 # number of generated data 
    dataset_path = '../data/{}.csv'.format(dataset_name)
    data = pd.read_csv(dataset_path)
    
    # generate samples
    samplesInit = generate_samples(data, sample_num)

    model=['knn','xgb','lr']

    # find key_attrs
    key_attrs = findKeyAttrs(num2cate(data), protect_attr)
    f = open(store_path + '{}_key.json'.format(dataset_name),'w')
    json.dump(key_attrs, f)
    
    for var in model:
        
        # set name
        model_name = dataset_name+"_"+var
        model_gene = ModelGene(model_name)
    
        # train model
        model, encoder, score = model_gene.fit_model(num2cate(data))
        
        # general samples
        num_samples, cate_samples = generate_model_samples(samplesInit, model, encoder) 
        
        # add the ID col 
        cate_samples.insert(loc=0, column='id', value= cate_samples.index)
        num_samples.insert(loc=0, column='id', value= num_samples.index)
        
        # save mdeol & samples to cache
        dataOut = pd.concat([num_samples,cate_samples])

        samples_path = os.path.join(store_path, '{}_samplesR.json'.format(model_name))
        cate_samples.to_json(samples_path, orient='records')

        samples_path = os.path.join(store_path, '{}_samplesR.csv'.format(model_name))
        cate_samples.to_csv(samples_path, index=False)

        samples_path = os.path.join(store_path, '{}_samples.json'.format(model_name))
        dataOut.to_json(samples_path, orient='records')

        samples_path = os.path.join(store_path, '{}_samples.csv'.format(model_name))
        dataOut.to_csv(samples_path, index=False)
        
        print(dataset_name + ' ' + var + ' accuracy: ' + str(score))
        
    print(dataset_name + ' all done')

# get_samples('bank','maritary')
get_samples('adult', 'sex')

samplePrior: Sample prior (min = 1.0) (java.lang.Double) [default:1.0]
structurePrior: Structure prior coefficient (min = 1.0) (java.lang.Double) [default:1.0]
faithfulnessAssumed: Yes if (one edge) faithfulness should be assumed (java.lang.Boolean) [default:true]
symmetricFirstStep: Yes if the first step step for FGES should do scoring for both X->Y and Y->X (java.lang.Boolean) [default:false]
maxDegree: The maximum degree of the graph (min = -1) (java.lang.Integer) [default:100]
verbose: Yes if verbose output should be printed or logged (java.lang.Boolean) [default:true]
numberResampling: The number of resampling iterations (min = 0) (java.lang.Integer) [default:0]
resampleSize: The resample size (min = 1) (java.lang.Integer) [default:1]
resamplingWithReplacement: Yes, if resampling with replacement (bootstrapping) (java.lang.Boolean) [default:true]
resamplingEnsemble: Ensemble method: Preserved (0), Highest (1), Majority (2) (java.lang.Integer) [default:1]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array w

adult knn accuracy: 0.8180561953017043


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array w

adult xgb accuracy: 0.8315676339628435


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array w

adult lr accuracy: 0.7962536465530478
adult all done


In [ ]:
# get_rules('dataTest','sex= Female', 'knn')
# # get_rules('dataTest','sex= Fema le', 'lr')
# get_rules('dataTest','sex= Female', 'knn')
# # get_rules('dataTest','sex= Fema le', 'lr')
# dataSets = ['dataTest','bank']
# models = ['xgb','knn','lr']
# print('start')
# for dataSet in dataSets:
#     for model in models:
#         sample_name = dataSet + '_' + model + '_samples_concat.json'
#         rule_name = dataSet + '_' + model + '_rules.json'
#         samples = pd.read_json(sample_name)

#         # add item id to rules
#         def item_within_rule(item, rule_context):
#             for attr_val in rule_context:
#                 attr, val = attr_val.split('=')
#                 if not item[attr] == val :
#                     return False
#             return True

#         rules = pd.read_json(rule_name)
#         rules['items'] = ''
#         for idx, rule in rules.iterrows():
#             rules.at[idx, 'items'] = [sample['id'] for i,sample in samples.loc[1000:1999].iterrows() if item_within_rule(sample, rule["antecedent"])] 
#         rules.to_json(rule_name, orient='records')

#         print(dataSet+'_'+model+' has done')

# print('All finished')